<div dir="rtl">
بسم‌الله الرحمن الرحیم
  
**امیرحسین بینائی شهری**  
تابستان ۱۴۰۴  

---

# 🎨 پروژه رسم با استفاده از انگشت و Mediapipe

در این پروژه از کتابخانه `Mediapipe` برای تشخیص دست و انگشتان، و از `OpenCV` برای رسم خطوط بر روی صفحه استفاده شده است.  
کاربر می‌تواند با حرکت انگشت اشاره و انگشت شصت، به صورت مجازی روی صفحه با رنگ‌ها و اندازه‌های مختلف نقاشی کند.

---

## ✨ امکانات اصلی

- تشخیص دو دست همزمان (دست راست: رسم / پاک‌کردن — دست چپ: انتخاب رنگ و سایز)
- رسم خطوط با انگشت اشاره و شصت
- تغییر رنگ رسم از میان:  
  سیاه، سفید، سبز، صورتی، قرمز، آبی، بنفش، زرد، قهوه‌ای، نارنجی
- تغییر سایز قلم با نوار لغزان (Slider)
- **پاک‌کن محلی** برای حذف بخش‌های خاص از مسیر
- **پاک‌کردن کل صفحه** با ورود به ناحیه "clear all"
- کلیدهای میان‌بر:
  - `z` = برگشت (Undo)
  - `y` = جلو رفتن (Redo)
  - `s` = ذخیره‌ی تصویر (Save)
  - `q` = خروج از برنامه (Quit)

---

## ⚙️ نحوه عملکرد

- زمانی که انگشت اشاره و شصت دست راست به هم نزدیک شوند، حالت رسم فعال می‌شود.
- اگر پاک‌کن محلی فعال باشد، همین حرکت باعث پاک‌کردن خطوط نزدیک می‌شود.
- با نزدیک‌کردن انگشت اشاره دست چپ به ناحیه‌های رنگی، رنگ قلم تغییر می‌کند.
- با نزدیک‌کردن به نوار اسلایدر، اندازه قلم تغییر خواهد کرد.

---

## 🧠 نکات فنی

- تشخیص دست با `mediapipe.solutions.hands` و دقت بالا (`min_detection_confidence=0.7`)
- رسم مسیرها با استفاده از `cv2.line`
- استفاده از `PIL` و `arabic_reshaper` برای نوشتن متن فارسی روی تصویر
- قابلیت بازنویسی خطوط با مدیریت مسیرهای ذخیره‌شده

---

## 📦 کتابخانه‌های مورد نیاز

```bash
pip install opencv-python mediapipe numpy pillow arabic_reshaper python-bidi
```

---

امید است این پروژه برای علاقه‌مندان به تعامل‌های تصویری و طراحی دیجیتال مفید باشد.  
پیشنهادات و بهبودها با آغوش باز پذیرفته می‌شود. ❤️
</div>

In [3]:
import cv2
import mediapipe as mp
import numpy
import math
from PIL import Image, ImageDraw, ImageFont
import arabic_reshaper
import bidi

def put_text_farsi(frame, text, org, color):
    frame_pillow = Image.fromarray(frame)
    text_reshap = arabic_reshaper.reshape(text)
    text_farsi = bidi.get_display(text_reshap)
    frame_font = ImageFont.truetype('amir.ttf', 20)
    pen = ImageDraw.Draw(frame_pillow)
    pen.text(org, text_farsi, color, frame_font)
    return numpy.array(frame_pillow)

# راه‌اندازی Mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7,max_num_hands=2)

# فعال‌سازی وب‌کم
cap = cv2.VideoCapture(0)

# مقادیر اولیه
brush_size = 10
slider_x = 580
slider_y = 150
slider_w = 20
slider_h = 250
slider_val = 100
jahat_dast = None
erasing_mode = False
was_in_eraser_zone = False



# ناحیه‌ها
trigger_zone = (540, 10, 620, 50) # ناحیه پاک‌کن (قرمز)
black_zone   = (495, 10, 535, 50) #     ناحیه رنگ سیاه
white_zone   = (450, 10, 490, 50) #ناحیه رنگ سفید
green_zone   = (405, 10, 445, 50) #ناحیه رنگ سبز
pink_zone    = (360, 10, 400, 50) #    ناحیه رنگ صورتی
red_zone     = (315, 10, 355, 50) #ناحیه رنگ قرمز
purpel_zone  = (270, 10, 310, 50) #ناحیه رنگ قرمز
blue_zone    = (225, 10, 265, 50) #      ناحیه رنگ آبی
yellow_zone  = (180, 10, 220, 50) #      ناحیه رنگ زرد
Brown_zone   = (135, 10, 175, 50) # ناحیه رنگ قهوه ایی
Orange_zone  = ( 90, 10, 130, 50) #   ناحیه رنگ نارنجی
eraser_zone  = (400, 10, 520, 50) #  ناحیه پاک کن محلی
erasing_mode = False              # پاک کن محلی غیر فعال است

# مسیر انگشت و وضعیت رسم
all_trails = []
current_trail = []
redo_stack = []    # redo

trail_color=(0,0,0) # رنگ پیش فرض نقاشی ، سیاه است
drawing = False     #      عملیات رسم غیر فعال است

#راه اندازی وبکم
while cap.isOpened():
    ret, frame = cap.read()
    frame_slider = frame.copy()
    if not ret:
        break
    frame_slider = cv2.flip(frame_slider, 1)
    frame        = cv2.flip(frame       ,  1)
    h, w, _ = frame.shape
    x_index, y_index = 0, 0  # مقدار اولیه

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)
    if result.multi_hand_landmarks:
        for hand_landmarks , d2 in zip(result.multi_hand_landmarks,result.multi_handedness):
            landmarks = hand_landmarks.landmark
            jahat_dast = d2.classification[0].label
            x_index  = int(landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * w)   # مختصات محور x انگشت اشاره
            y_index  = int(landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * h)   # مختصات محور y انگشت اشاره

            x_THUMB = int(landmarks[mp_hands.HandLandmark.THUMB_TIP].x * w)           # مختصات محور x انگشت شصت
            y_THUMB = int(landmarks[mp_hands.HandLandmark.THUMB_TIP].y * h)           # مختصات محور y انگشت شصت

            distance = math.hypot(x_index - x_THUMB, y_index - y_THUMB) # محاسبه فاصله انگشت اشاره و شصت بر حست تعداد پیکسل


            if distance < 40 and  jahat_dast=='Right':
                if erasing_mode:
                    # پاک کردن مسیرها در نزدیکی انگشت
                    updated_trails = []
                    for trail, color, size in all_trails:
                        split_trail = []
                        current_segment = []

                        for point in trail:
                            dist = math.hypot(point[0] - x_index, point[1] - y_index)
                            if dist > 20:
                                current_segment.append(point)
                            else:
                                if len(current_segment) > 1:
                                    split_trail.append(current_segment)
                                current_segment = []

                        # اضافه کردن آخرین قطعه اگر معتبر بود
                        if len(current_segment) > 1:
                            split_trail.append(current_segment)

                        for segment in split_trail:
                            updated_trails.append((segment, color, size))

                    all_trails = updated_trails

                    continue  # چون داریم پاک می‌کنیم، دیگه ادامه نمی‌دیم برای رسم
                else:
                    if not drawing:
                        drawing = True
                        current_trail = []
                    current_trail.append((x_index, y_index))

            else:
                if drawing:
                    drawing = False
                    if current_trail:
                        all_trails.append((current_trail, trail_color, brush_size))  # ذخیره همراه رنگ
                        current_trail = []



    # اگر انگشت وارد ناحیه پاک‌کن شد، پاک می‌کند مسیرها و رنگ سیاه را خاموش می‌کند
    x1, y1, x2, y2 = trigger_zone
    if  x1 <= x_index <= x2 and y1 <= y_index <= y2 and jahat_dast=='Right':
        all_trails = []
        current_trail = []
        erasing_mode = False

    # اگر انگشت وارد ناحیه رنگ سیاه شد، حالت رنگ سیاه فعال شود و برای همیشه بماند
    bx1, by1, bx2, by2 = black_zone
    if jahat_dast == 'Left' and bx1 <= x_index <= bx2 and by1 <= y_index <= by2:
        trail_color = (0,0,0)
        erasing_mode = False

      #اگر انگشت وارد ناحیه رنگ سفید شد، حالت رنگ سفید فعال شود و برای همیشه بماند
    wx1, wy1, wx2, wy2 = white_zone
    if jahat_dast == 'Left' and wx1 <= x_index <= wx2 and wy1 <= y_index <= wy2:
        trail_color = (255, 255, 255)
        erasing_mode = False

    #اگر انگشت وارد ناحیه رنگ سبز شد، حالت رنگ سبز فعال شود و برای همیشه بماند
    gx1, gy1, gx2, gy2 = green_zone
    if jahat_dast == 'Left' and  gx1 <= x_index <= gx2 and gy1 <= y_index <= gy2:
        trail_color = (0, 255, 0)
        erasing_mode = False

    #اگر انگشت وارد ناحیه رنگ صورتی شد، حالت رنگ صورتی فعال شود و برای همیشه بماند
    px1, py1, px2, py2 = pink_zone
    if jahat_dast == 'Left' and px1 <= x_index <= px2 and py1 <= y_index <= py2:
        trail_color = (147, 20, 255)
        erasing_mode = False

    # اگر انگشت وارد ناحیه رنگ قرمز شد، حالت رنگ قرمز فعال شود و برای همیشه بماند
    rx1, ry1, rx2, ry2 = red_zone
    if jahat_dast == 'Left' and rx1 <= x_index <= rx2 and ry1 <= y_index <= ry2:
        trail_color = (0, 0, 255)
        erasing_mode = False

    # اگر انگشت وارد ناحیه رنگ بنفش شد، حالت رنگ بنفش فعال شود و برای همیشه بماند
    pux1, puy1, pux2, puy2 = purpel_zone
    if jahat_dast == 'Left' and pux1 <= x_index <= pux2 and puy1 <= y_index <= puy2:
        trail_color = (128, 0, 128)
        erasing_mode = False

    # اگر انگشت وارد ناحیه رنگ آبی شد، حالت رنگ آبی فعال شود و برای همیشه بماند
    bluex1, bluey1, bluex2, bluey2 = blue_zone
    if jahat_dast == 'Left' and bluex1 <= x_index <= bluex2 and bluey1 <= y_index <= bluey2:
            trail_color = (255, 0, 0)
            erasing_mode = False

    # اگر انگشت وارد ناحیه رنگ زرد شد، حالت رنگ زرد فعال شود و برای همیشه بماند
    yx1, yy1, yx2, yy2 = yellow_zone
    if jahat_dast == 'Left' and yx1 <= x_index <= yx2 and yy1 <= y_index <= yy2:
        trail_color = (0, 255, 255)
        erasing_mode = False

    # اگر انگشت وارد ناحیه رنگ قهوه ایی شد، حالت رنگ قهوه ایی فعال شود و برای همیشه بماند
    Bx1, By1, Bx2, By2 = Brown_zone
    if jahat_dast == 'Left' and Bx1 <= x_index <= Bx2 and By1 <= y_index <= By2:
            trail_color = (19, 69, 139)
            erasing_mode = False

    # اگر انگشت وارد ناحیه رنگ نارنجی شد، حالت رنگ نارنجی فعال شود و برای همیشه بماند
    nx1, ny1, nx2, ny2 = Orange_zone
    if jahat_dast == 'Left' and nx1 <= x_index <= nx2 and ny1 <= y_index <= ny2:
        trail_color = (0, 165, 255)
        erasing_mode = False
    # پاک کن محلی
    ex1, ey1, ex2, ey2 = eraser_zone
    in_eraser_zone = ex1 <= x_index <= ex2 and ey1 <= y_index <= ey2 and  jahat_dast=='Right'
    if in_eraser_zone and not was_in_eraser_zone:
        erasing_mode = not erasing_mode  # تغییر وضعیت پاک‌کن (toggle)
    was_in_eraser_zone = in_eraser_zone

    # به‌روزرسانی سایز قلم
    brush_size = max(1, int((slider_val / slider_h) * 50))

    # رسم نوار لغزان در پنجره‌ی کنترل
    cv2.rectangle(frame_slider, (slider_x, slider_y), (slider_x + slider_w, slider_y + slider_h), (200, 200, 200), -1)
    cv2.rectangle(frame_slider, (slider_x, slider_y), (slider_x + slider_w, slider_y + slider_val),  trail_color, -1)
    cv2.putText(frame_slider, f'Brush Size: {brush_size}', (slider_x-85, slider_y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)


    # بررسی برخورد با اسلایدر
   # if jahat_dast == 'Left':
    if jahat_dast == 'Left' and slider_x <= x_index <= slider_x + slider_w and \
            slider_y <= y_index <= slider_y + slider_h:
        slider_val = y_index - slider_y


    # رسم مسیرهای قبلی
    for trail, color, size in all_trails:
        for i in range(1, len(trail)):
             cv2.line(frame, trail[i - 1], trail[i], color, size)

    # رسم مسیر فعلی
    for i in range(1, len(current_trail)):
         cv2.line(frame, current_trail[i - 1], current_trail[i], trail_color,  brush_size)

    # رسم ناحیه‌ها
    cv2.rectangle(frame, (trigger_zone[0], trigger_zone[1]),(trigger_zone[2],trigger_zone[3]), (0, 0, 255), 2)
    #frame = put_text_farsi(frame, "clear_all", (trigger_zone[0] + 10, trigger_zone[1] + 5), (0, 0, 0))
    cv2.putText(frame, "clear_all", (trigger_zone[0] + 1, trigger_zone[1] + 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

    cv2.rectangle(frame_slider, (black_zone[0] , black_zone[1]) , (black_zone[2], black_zone[3])  , (  0,   0,   0), -1)

    cv2.rectangle(frame_slider, (white_zone[0] , white_zone[1]) , (white_zone[2], white_zone[3])  , (255, 255, 255), -1)

    cv2.rectangle(frame_slider, (green_zone[0] , green_zone[1]) , (green_zone[2], green_zone[3])  , (  0, 255,   0), -1)

    cv2.rectangle(frame_slider, (pink_zone[0]  , pink_zone[1])  , (pink_zone[2] , pink_zone[3])   , (147,  20, 255), -1)

    cv2.rectangle(frame_slider, (red_zone[0]   , red_zone[1])   , (red_zone[2]  , red_zone[3])    , (  0,   0, 255), -1)

    cv2.rectangle(frame_slider, (purpel_zone[0], purpel_zone[1]), (purpel_zone[2], purpel_zone[3]), (128,   0, 128), -1)

    cv2.rectangle(frame_slider, (blue_zone[0]  , blue_zone[1])  , (blue_zone[2]  , blue_zone[3])  , (255,   0,   0), -1)

    cv2.rectangle(frame_slider, (yellow_zone[0], yellow_zone[1]), (yellow_zone[2], yellow_zone[3]), (  0, 255, 255), -1)

    cv2.rectangle(frame_slider, (Brown_zone[0] , Brown_zone[1]) , (Brown_zone[2] , Brown_zone[3]) , ( 19,  69, 139), -1)

    cv2.rectangle(frame_slider, (Orange_zone[0], Orange_zone[1]), (Orange_zone[2], Orange_zone[3]), (  0, 165, 255), -1)


    # اگر پاک‌کن فعاله، مستطیل پر رنگ تر (قرمز پر) رسم بشه
    if erasing_mode:
        cv2.rectangle(frame, (eraser_zone[0], eraser_zone[1]), (eraser_zone[2], eraser_zone[3]), (0, 0, 200), -1)
    else:
        cv2.rectangle(frame, (eraser_zone[0], eraser_zone[1]), (eraser_zone[2], eraser_zone[3]), (100, 100, 100), 2)
    # نوشتن متن روی مستطیل، در هر دو حالت
    cv2.putText(frame, "local_eraser", (eraser_zone[0] + 1, eraser_zone[1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255) if erasing_mode else (0, 0, 0), 2)



    if jahat_dast == 'Right':
     cv2.circle(frame, (x_index, y_index),  brush_size , trail_color, -1) # نقطه انگشت

    if jahat_dast == 'Left':
         cv2.circle(frame_slider, (x_index, y_index), brush_size, trail_color, -1)  # نقطه انگشت




         # نمایش کادر دور رنگ فعال
    # دیکشنری رنگ‌ها به نواحی مربوطه
    color_zone_map = {
        (0, 0, 0): black_zone,
        (255, 255, 255): white_zone,
        (0, 255, 0): green_zone,
        (147, 20, 255): pink_zone,
        (0, 0, 255): red_zone,
        (128, 0, 128): purpel_zone,
        (255, 0, 0): blue_zone,
        (0, 255, 255): yellow_zone,
        (19, 69, 139): Brown_zone,
        (0, 165, 255): Orange_zone
    }
    if trail_color in color_zone_map:
        zone = color_zone_map[trail_color]
        # رسم کادر دور ناحیه رنگ فعال
        cv2.rectangle(frame_slider, (zone[0], zone[1]), (zone[2], zone[3]), (0, 0, 0), 2)

    status_text = f"Color: {trail_color} | Size: {brush_size} | Mode: {'Erasing' if erasing_mode else 'Drawing'}"
    cv2.putText(frame_slider, status_text, (10, h - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (trail_color), 2)

    cv2.imshow("Control Panel", frame_slider)
    cv2.imshow("Air Drawing", frame)

    key = cv2.waitKey(1) & 0xFF



    # Undo
    if key == ord('z'):
        if all_trails:
            redo_stack.append(all_trails.pop())

    # Redo
    elif key == ord('y'):
        if redo_stack:
            all_trails.append(redo_stack.pop())


    elif key == ord('s'):  # Save
        cv2.imwrite('drawing.png', frame)
    elif key == ord('q') or cv2.getWindowProperty('Air Drawing', cv2.WND_PROP_VISIBLE) < 1:
        break

cap.release()
cv2.destroyAllWindows()